In [ ]:
# 수치형 데이터
# 유저 특징 정보 추출 
# userfeaturesextraction
from itertools import islice

profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')

profile_df = profile_df.set_index('profile_id') # profile_id를 인덱스로 지정
user_features = profile_df[['age']].to_dict() # profile_df의 age를 dictionary로 변형
print("user_id 3의 age 정보 :", user_features['age'][3]) # user_id와 age를 엮어주는거 같음, 
# user_features라는 딕셔너리 안의 key값은 age, age키의 값이 profile_id를 key로 가지고 나이를 값으로 가지는 딕셔너리가 됨 (user_features는 이중 딕셔너리)

dict(islice(user_features['age'].items(), 5)) # 딕셔너리 인덱스 5까지 출력

In [ ]:
# 범주형 데이터
# 아이템 특징 정보 추출 
# itemfeaturesextraction
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')

meta_df = meta_df.set_index('album_id')

# 범주형 데이터를 수치형 데이터로 변경 
le = LabelEncoder()
meta_df['genre_mid'] = le.fit_transform(meta_df['genre_mid'])
item_features = meta_df[['genre_mid']].to_dict()
print("album_id 749의 genre_mid 정보 :", item_features['genre_mid'][749])

In [ ]:
# 11
# 추출한 특징 정보의 속성을 저장
# savegenremidnunique
cfg.n_genres = meta_df['genre_mid'].nunique() # <-- 범주형 데이터는 특징 속성을 저장해야 함
cfg.n_sex = profile_df['sex'].nunique()
cfg.n_continuous_feats = 1 

In [ ]:
        self.user_embedding_mf = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim) # n_users = data.profile_id.max()+1, emb_dim = 256
        self.item_embedding_mf = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim)
        # ex) embedding_dim이 3, num_embeddings=8이면 // num_embeddings 행 개수, embedding_dim이 열 개수
        
        # tensor([[-0.1778, -1.9974, -1.2478],
        # [ 0.0000,  0.0000,  0.0000],
        # [ 1.0921,  0.0416, -0.7896],
        # [ 0.0960, -0.6029,  0.3721],
        # [ 0.2780, -0.4300, -1.9770],
        # [ 0.0727,  0.5782, -3.2617],
        # [-0.0173, -0.7092,  0.9121],
        # [-0.4817, -1.1222,  2.2774]], requires_grad=True)
        
        self.user_embedding_mlp = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim)
        self.item_embedding_mlp = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim) # 여기까지 4줄은 다 똑같음
                
        self.genre_embeddig = nn.Embedding(num_embeddings=self.n_genres, embedding_dim=self.n_genres//2) # 장르 임베딩 하이퍼 파라미터는 n_genres = meta_df['genre_mid'].nunique()
                                                                                                        # 임베딩 차원이 장르/2라는 특이한 점이 있음
        self.sex_embeddig = nn.Embedding(num_embeddings=self.n_sex, embedding_dim=self.n_sex//2) # 추가한거   

        self.mlp_layers = nn.Sequential(
            nn.Linear(2*self.emb_dim + self.n_genres//2 + self.n_sex//2 + self.n_continuous_feats, self.layer_dim), # (https://m.blog.naver.com/fbfbf1/222480437930)
            # 추가한거
            # nn.Linear의 형태는 nn.Linear(input_dim,output_dim)이다
            # 여기서 input은 256*2 + 29//2 + 1, output은 256
            nn.ReLU(), 
            nn.Dropout(p=self.dropout), # dropout = 0.05
            nn.Linear(self.layer_dim, self.layer_dim//2), # layer_dim = 256
            nn.ReLU(), 
            nn.Dropout(p=self.dropout)